# Large Language Models

## **Session 4-1:** Introduction to LLMs

<img src="../figures/LLM_sizes.png" alt="fig" width="300" align="right" style="padding: 30px;" />

Large Language Models (LLMs) are:  
* Deep-learning-based models for natural language
* _Large_ because:
  * Model sizes: $10^9 .. 10^{12}$ Parameters
  * Training copus: $10^{11} ... 10^{13}$ tokens
  * Training _compute_: $10^{21} ... 10^{25}$ Floating Point Operations (FLOPS)
* very resource hungry to train
* Example $10^{22}$ FLOPS:
    * on i7 7700K CPU with $241\cdot 10{9}$ FLOPS/s: 1300 years  
    * Nvidia RTX 4090 consumer GPU with $82\cdot 10^{12}$ FLOPS: 3.9 years
    * El Captain cluster (top cluster): $1.47 \cdot 10^{18}$ FLOPS: 2h
* __but note:__ this is only compute; data transfer, and memory size is a major bottleneck!
* Training an LLM from scratch is in the order of the entire budget of a medium-sized university 



### The Transformer

<img src="../figures/Transformer.png" alt="fig" width="300" align="right" style="padding: 30px;" />


* The modern language models (Llama, GPT, ...) are all based on the transformer architecture. 
    * GPT: generative pretrained __Transformer__
* The _attention_ enables the model capture relationship of tokens/words throughout the input 
* _multi-headed attention_ refers to multiple (parallel) attention heads with seperate (learned) parameters
    * different heads learn different relationships


* The main parts are the encoder (left) and decoder (right). Some modern models as e.g. Phi-4 are decoder-only. 

* Note: see also the paper [Attention Is All You Need](https://doi.org/10.48550/arXiv.1706.03762).




<!-- 

generative pretraining: first part of training with unlabeled text (predict next token)
after first add and norm: residual connection 
decoder only: text  generation; encoder only: classifications and embedding but not as widely used

-->


### Terms for LLMs: 

* __Token__: input and output of the model is not on the character level but token level.
  * Tokenization is translating the input string into tokens and the output tokens into a string
  * Typically, tokens are a mixture of words, sub-words, or single characters
  * tokenizer "vocabulary" is model specific, see e.g. [here](https://huggingface.co/deepseek-ai/DeepSeek-R1/raw/main/tokenizer.json) at entry "vocab".
  <!--* Rule of thumb: on average 100 tokens ~ 75 words-->
* __context window__: The maximum number of tokens the model can consider for an output.  
* __Model sizes__: 8B --> $8$ Billion parameters
* __Quantization__: reduces the number of bits used to represent the model parameters
    * is applied after training to reduce required memory and increase inference tokens
* __Inference__: the process by which the (already trained) machine learning model obtains output from given input
* __Multimodal__ model: models which support different types of input data, e.g. text, images, and audio.
* __prompt__: The full input text given to the LLM.
    * The prompt might include instructions, examples, and a task/question 
    * __query__: The specific task/question provided; part of the prompt
    * __context__: Additional information included in the prompt to guide the model, such as examples, prior , conversations, or task-specific instructions
  <!--* query: part of the prompt-->
  


In [1]:
# Example Tokenization: 
from transformers import BertTokenizer
# Initialize the tokenizer from Bert -> an older Language model from 2018. 
# On Hugging Face (provides transformers library) the tokenizer.json is provided together with the files: 
# https://huggingface.co/Qwen/Qwen2.5-VL-32B-Instruct/blob/main/tokenizer.json for Qwen2.5
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# Example sentence: multibody is not a single token but 2. 
sentence = "This is a simple sentence, showing tokenization at the Multibody Dynamics Workshop 2025 in Innsbruck. "
#sentence = "Welcome to my multibody dynamics talk. "
print(f'Sentence: "{sentence}"')

# 1. Tokenization: Convert the sentence into tokens
tokens = tokenizer.tokenize(sentence)
print(f"Tokens: {tokens}")  # Output the tokens

# 2. Convert tokens to input IDs (numerical representation)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Input IDs: {input_ids}") # Output the input_ids


Sentence: "This is a simple sentence, showing tokenization at the Multibody Dynamics Workshop 2025 in Innsbruck. "
Tokens: ['This', 'is', 'a', 'simple', 'sentence', ',', 'showing', 'token', '##ization', 'at', 'the', 'Multi', '##body', 'Dynamics', 'Workshop', '202', '##5', 'in', 'Inn', '##s', '##bruck', '.']
Input IDs: [1188, 1110, 170, 3014, 5650, 117, 4000, 22559, 2734, 1120, 1103, 18447, 14637, 25082, 16350, 17881, 1571, 1107, 9859, 1116, 27824, 119]


### Prompt Engineering
<!--span style="font-size: 16pt"-->  

How prompts are formulated has a major influence on the model’s output and its quality.
* Simple prompt: "Create a simulation model of a spring-damper system."  
      vs.
* detailed prompt: "Consider a mass–spring-damper system with the following properties: mass m = 8 kg, stiffness k = 5000 N/m, and damping d = 50 Ns/m. The force applied to the mass is f = 100 N. Create a simulation model using Exudyn to simulate the dynamics of the mass–spring-damper system. The spring has a length of 5 cm and is relaxed in the initial position. Please write the code with no comments and in one block." 
* Creating _good_ prompts is typically heuristic
* __System prompts__ (provided to the model by default) guide general behaviour, e.g:
  * "You are a helpful assistant. "
  * "You are an AI assistant answering questions in the domain of mechanical engineering and an expert in multibody dynamics. "

These instructions heuristically improve the quality of responses. 

<!--/span-->


### Training Corpus

    
LLMs are trained on a large amount of data
* It is generally not known exactly on which data the model is trained.
* GPT3.5 is trained on Wikipedia, Books, and "Webcrawl"
* Model Performance depends strongly on data quality
* Typically, GitHub is also part of the training corpus
    * But: knowledge cut-off due to training date



### How LLMs gain new knowledge: 
    
1. Training from __scratch__ and include new data
2. __Finetuning__: further training of (pre-trained) model
    * __Catastrophic forgetting__ might happen: the LLM forgets previously known facts/tasks ´
    * Parameter efficient fine tuning (PEFT): not the whole model is retrained, but only parts of it
    * [Low Rank Adaptation (LORA)](https://arxiv.org/abs/2106.09685): initial model parameters are frozen and additional _adapters_ (low-rank matrices) are added to the model and trained 
4. Retrievel Augmented Generation (__RAG__):
    * A __database__ is provided: e.g. as pdf files from books, publications or code documentation 
    * A __retriever__ uses either keyword matching or neural embeddings to find semantically similar documents and returns k most relevant documents
    * The __prompt__ is processed together with retrieved documents

5. In-Context-Learning:
    * LLMs learn during inference  by using examples or instructions previously provided through the input without updating the model's weights
    * No explicit _training_ is performed


### Availible Models

__Disclaimer__: LLMs are a fast-paced topic with new libraries and models coming up often. 
__Note__: Most companies develop both large flagship models to push SOTA and smaller, more efficient models

Proprietory:   
* OPEN AI: GPT4o, GPT-4.1, GPT-4.1-mini, o1; see also the [model list](http://platform.openai.com/docs/models). 
* Google: Gemini 2.0 Flash, 2.5 Pro
* Anthropic: Claude 3.5 Haiku, Claude 3.7 Sonnet

Open (weights):  
* Meta: Llama Models
  * [Llama 2](https://www.llama.com/llama2/)/[Llama 3](https://www.llama.com/models/llama-3/): 8B, 70B, 405B in sizes of 8B, 70B, 405B
  * [Llama 4](https://ai.meta.com/blog/llama-4-multimodal-intelligence/): mixture of expert models with up to 10M context length;  _Scout_: 109B/17B active parameters
  * Note: for Llama models, an authorization request is required
* Microsoft:
    *   [Phi-4](https://huggingface.co/microsoft/phi-4), and Phi-3.   
* Deepseek:
  * [Deepseek-R1](https://arxiv.org/abs/2501.12948): strong reasoning Model
  * [DeepSeek-Coder](https://arxiv.org/abs/2401.14196)
* Mistral AI:
  * Mistral
  * Codestral
* Alibaba AI:
  * Qwen2.5 32B/72B, Qwen-Coder
 
For many models - even proprietary ones - technical report _papers_ are available. 



### Quantization: 


Besides compute, the available memory limits the speed of LLM inference. 
Inference performance of LLMs often only degrades slightly by using parameters down to 4-bit precision see e.g. [Dettmers, Zettlemoyer: the case for 4-bit precision: k-bit Inference Scaling Laws, 2023](https://doi.org/10.48550/arXiv.2212.09720). 

For Llama3, 70B, the model size in the memory is:  
* $70*4 = 280GB$ in 32-bit floating point precision (also called fp32 or single-precision)
* $70*0.5 = 35GB$ in 4bit precision
  * this is denoted as Llama3 70B __Q4__

Modern GPU hardware often supports larger FLOPS/s for lower precision, resulting in higher throughput.   
__Note__: A 70B-Q4 model requires 35GB in memory, while an 8B model in fp32 is 32GB. 




### Libraries: 


__transformers__:  
Provided by _Hugging Face_, transformers is a Python library of pretrained natural language processing, computer vision, audio, and multimodal models

__gpt4all__:  
Provides a python library and graphics interface. Includes simple indexing of local documents with RAG. 

__llamacpp__:  
An efficient C++ library for LLM inference, which also includes a Python interface.  
_Note_: although installable with <code> pip install llama-cpp-python </code> a C++ compiler is required for installation. 


__bitsandbytes__:  
Provides local quantization, used e.g. in transformers. 

__torch__ and __tensorflow__ are general ML libraries typically running in the backend. 




### Session Content: 

* [4-2_Inspect_Local_Model](4-2_Inspect_Local_Model.ipynb): Load a local model and inspect its properties. 
* [4-3_CreateOscillator](4-3_CreateOscillator.ipynb): Create the simulation code for a simple Oscillator using a local model.  
* [4-4_LLM_API](4-4_LLM_API.ipynb): Create a multibody model of a Slider-Crank using context learning and the (commercial) OpenAI online API. 
* [4-5_Optional_CreateMultibodyModel](4-5_Optional_CreateMultibodyModel.ipynb):  Create a Slider-Crank model using the open-source code Exudyn using a local model. Disclaimer: This requires more resources and parts use llama_cpp, which is only for demonstration. 

